Autor: `Luan Gabriel Silva Oliveira`



**importações das bibliotecas**

*   **os:** uma biblioteca para interagir com o sistema operacional, como a criação de diretórios ou a verificação de arquivos existentes.
numpy: uma biblioteca para trabalhar com arrays multidimensionais e realizar operações matemáticas.
*   **IPython.display:** uma biblioteca para exibir conteúdo dentro do notebook Jupyter, como imagens e vídeos.
*   **json:** uma biblioteca para trabalhar com arquivos JSON, um formato de dados bastante utilizado em aplicações web.
*   **torch:** uma biblioteca para criação e treinamento de modelos de aprendizado de máquina baseados em redes neurais.
*   **uuid:** uma biblioteca para gerar identificadores únicos universais.
*   **PIL:** uma biblioteca para processamento de imagens, incluindo abertura e manipulação de arquivos de imagem.


In [1]:
import os
import numpy as np
import warnings
import pandas as pd
from IPython.display import display, Image
import matplotlib.pyplot as plt
import json
import torch as th
import random
import uuid
from sklearn.model_selection import train_test_split
from sklearn import metrics
from pandas_profiling import ProfileReport
from PIL import Image

Código responsável por permitir que os usuários carreguem dados externos para o ambiente de notebook do Google Colaboratory para análise de dados ou treinamento de modelos de aprendizado de máquina.

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


criando um diretório .kaggle na pasta principal do usuário e movendo um arquivo chamado kaggle.json para dentro deste diretório.

In [3]:
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Baixando o conjunto de dados do Kaggle chamado "FruitNet - Indian Fruits Dataset with Quality" do Kaggle para o ambiente de notebook ou para a máquina local do usuário

In [4]:
!kaggle datasets download -d shashwatwork/fruitnet-indian-fruits-dataset-with-quality

fruitnet-indian-fruits-dataset-with-quality.zip: Skipping, found more recently modified local copy (use --force to force download)


Descompactando um arquivo zip baixado anteriormente do Kaggle, permitindo que o usuário acesse e trabalhe com os arquivos descompactados em seu ambiente de notebook ou em seu computador local.

In [5]:
from zipfile import ZipFile
file_name = "/content/fruitnet-indian-fruits-dataset-with-quality.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


Importando bibliotecas úteis para processamento de imagens.

*   A biblioteca "cv2", que é a interface Python para a biblioteca OpenCV (Open Source Computer Vision Library), uma biblioteca de código aberto que inclui várias funções para processamento de imagens e visão computacional.
*   A classe "tqdm_notebook" da biblioteca "tqdm", que fornece uma barra de progresso interativa para iterações em loops.
*   As bibliotecas "os" e "glob". A biblioteca os fornece uma maneira de interagir com o sistema operacional, enquanto a biblioteca glob fornece uma maneira de listar arquivos de um determinado diretório que correspondam a um padrão específico.






In [6]:
import tensorflow as tf
import os,glob
import cv2
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

<ipython-input-6-dd7e21e0c7f3>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook as tqdm


In [7]:
bad_path = '/content/Processed Images_Fruits/Bad Quality_Fruits'
good_path = '/content/Processed Images_Fruits/Good Quality_Fruits'

Lendo imagens de frutas de dois diretórios diferentes, "bad_path" e "good_path", e criando um DataFrame do pandas que armazena os caminhos das imagens de frutas e seus respectivos rótulos. Isso é útil para preparar um conjunto de dados para treinar um modelo de classificação de imagens de frutas.preparando um conjunto de dados para treinar um modelo de classificação de imagens de frutas e realizar pré-processamento nas imagens para ajustá-las para o modelo.

In [8]:
file_paths=[]
labels=[]
ht=0
wt=0
samples=0
sample_count=20
for quality in [bad_path, good_path]:    
    fruit_list=os.listdir(quality)
    for fruit in fruit_list:
        fruit_path=os.path.join(quality, fruit)
        img_list=os.listdir(fruit_path)
        for i, img in enumerate(img_list):
            img_path=os.path.join(fruit_path, img)
            if i < sample_count:
                img=plt.imread(img_path)               
                ht += img.shape[0]
                wt += img.shape[1]
                samples +=1
            file_paths.append(img_path)
            labels.append(str.lower(fruit))
Fseries = pd.Series(file_paths, name='image')
Lseries = pd.Series(labels, name='label')
df=pd.concat([Fseries, Lseries], axis=1)

Dividino o DataFrame df em três conjuntos: treinamento, teste e validação, com base em uma proporção definida pelos valores das variáveis trsplit e vsplit. Também realiza algumas análises sobre o conjunto de dados. O código exibe o tamanho de cada conjunto de dados e o número de imagens em cada classe no conjunto de treinamento. Além disso, o código calcula a altura média, a largura média e a relação de aspecto média de todas as imagens no conjunto de dados. 

Essas informações podem ser usadas para redimensionar ou ajustar as imagens antes de alimentá-las ao modelo de classificação.

In [9]:
trsplit=.9
vsplit=.05
dsplit =vsplit/(1-trsplit)
strat=df['label']
train_df, dummy_df=train_test_split(df, train_size=.9, shuffle=True, stratify=strat)
strat=dummy_df['label']
test_df, valid_df=train_test_split(dummy_df, train_size=dsplit, shuffle=True, stratify=strat)
print('train_df lenght: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))
classes=list(train_df['label'].unique())
class_count = len(classes)
groups=df.groupby('label')
print('{0:^30s} {1:^13s}'.format('CLASS', 'IMAGE COUNT'))
for label in train_df['label'].unique():
      group=groups.get_group(label)
      samples=len(group)
      print('{0:^30s} {1:^13s}'.format(label, str(len(group))))
wave=wt/samples
have=ht/samples
aspect_ratio= have/wave
print ('Average Image Height: ' ,have, '  Average Image Width: ', wave, '  Aspect ratio: ', aspect_ratio)

train_df lenght:  16606   test_df length:  923   valid_df length:  923
            CLASS               IMAGE COUNT 
       pomegranate_good            5940     
         banana_good               1113     
       pomegranate_bad             1187     
          apple_good               1149     
          apple_bad                1141     
          guava_bad                1129     
           lime_bad                1085     
         orange_good               1216     
          lime_good                1094     
          orange_bad               1159     
          guava_good               1152     
          banana_bad               1087     
Average Image Height:  54.991720331186755   Average Image Width:  52.40110395584177   Aspect ratio:  1.049438202247191


In [ ]:
plt.plot(wave)

Definindo uma função chamada "trim" que recebe o DataFrame, faz a cópia desse DataFrame a fim de evitar a modificação dos dados originais. Essa função concatena os subgrupos, redefinindo o índice e retorna o DataFrame modificado.

In [10]:
def trim (df, max_size, min_size, column):
    df=df.copy()
    original_class_count= len(list(df[column].unique()))
    print ('Original Number of classes in dataframe: ', original_class_count)
    sample_list=[] 
    groups=df.groupby(column)
    for label in df[column].unique():        
        group=groups.get_group(label)
        sample_count=len(group)         
        if sample_count> max_size :
            strat=group[column]
            samples,_=train_test_split(group, train_size=max_size, shuffle=True, stratify=strat)            
            sample_list.append(samples)
        elif sample_count>= min_size:
            sample_list.append(group)
    df=pd.concat(sample_list, axis=0).reset_index(drop=True)
    final_class_count= len(list(df[column].unique())) 
    if final_class_count != original_class_count:
        print ('*** WARNING***  dataframe has a reduced number of classes' )
    balance=list(df[column].value_counts())
    print (balance)
    return df

In [11]:
train_df

,image,label
10573,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good
16119,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good
17890,/content/Processed Images_Fruits/Good Quality_...,banana_good
4501,/content/Processed Images_Fruits/Bad Quality_F...,pomegranate_bad
5193,/content/Processed Images_Fruits/Bad Quality_F...,pomegranate_bad
...,...,...
9232,/content/Processed Images_Fruits/Good Quality_...,lime_good
6308,/content/Processed Images_Fruits/Bad Quality_F...,apple_bad
10716,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good
12072,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good


In [12]:
dataset_concated = pd.concat([train_df, test_df])

In [13]:
dataset_concated

,image,label
10573,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good
16119,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good
17890,/content/Processed Images_Fruits/Good Quality_...,banana_good
4501,/content/Processed Images_Fruits/Bad Quality_F...,pomegranate_bad
5193,/content/Processed Images_Fruits/Bad Quality_F...,pomegranate_bad
...,...,...
15770,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good
5624,/content/Processed Images_Fruits/Bad Quality_F...,pomegranate_bad
12064,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good
13553,/content/Processed Images_Fruits/Good Quality_...,pomegranate_good


Criando um modelo de rede neural convolucional utilizando a arquitetura InceptionV3 pré-treinada na base de dados ImageNet para classificar imagens em duas categorias, usando o conjunto de dados preparado anteriormente.
Define-se o modelo, que é composto pelo modelo base InceptionV3, com suas camadas congeladas e sem a camada de classificação final. Em seguida, o modelo é compilado com o otimizador Adam, a função de perda binary_crossentropy e a métrica accuracy. Por fim, o modelo é treinado com os dados de treino e validação usando o método fit() e a variável history armazena os valores de perda e métricas de acurácia durante o treinamento. Em seguida, o modelo é avaliado com os dados de validação usando o método evaluate() e os resultados de perda e acurácia são impressos na tela.

In [14]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Importar as bibliotecas necessárias

# 2. Preparar o conjunto de dados
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    directory=None,
                                                    x_col="image",
                                                    y_col="label",
                                                    subset="training",
                                                    batch_size=32,
                                                    target_size=(299, 299))

validation_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                         directory=None,
                                                         x_col="image",
                                                         y_col="label",
                                                         subset="validation",
                                                         batch_size=32,
                                                         target_size=(299, 299))

# 3. Preparar o modelo
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# 4. Treinar o modelo
history = model.fit(train_generator, epochs=1, validation_data=validation_generator)

# 5. Avaliar o modelo
loss, accuracy = model.evaluate(validation_generator)
print("Validation loss:", loss)
print("Validation accuracy:", accuracy)

Found 13285 validated image filenames belonging to 12 classes.
Found 3321 validated image filenames belonging to 12 classes.
104/104 [==============================] - 835s 8s/step - loss: 0.2876 - accuracy: 0.9167
Validation loss: 0.2876403033733368
Validation accuracy: 0.9166662096977234
